## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [11]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([784, 128], stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([128]))
        self.W2 = tf.Variable(tf.random.normal([128, 10], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([10]))
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 784])  # Flatten
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [5]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [14]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 1.5623729 ; accuracy 0.57496667
epoch 1 : loss 1.5567958 ; accuracy 0.5772667
epoch 2 : loss 1.5512521 ; accuracy 0.57985
epoch 3 : loss 1.5457412 ; accuracy 0.58205
epoch 4 : loss 1.5402632 ; accuracy 0.58428335
epoch 5 : loss 1.5348176 ; accuracy 0.58648336
epoch 6 : loss 1.5294043 ; accuracy 0.5891
epoch 7 : loss 1.5240232 ; accuracy 0.5911667
epoch 8 : loss 1.5186743 ; accuracy 0.5933
epoch 9 : loss 1.5133568 ; accuracy 0.59533334
epoch 10 : loss 1.508071 ; accuracy 0.59711665
epoch 11 : loss 1.5028169 ; accuracy 0.59926665
epoch 12 : loss 1.4975942 ; accuracy 0.60171664
epoch 13 : loss 1.4924026 ; accuracy 0.60385
epoch 14 : loss 1.4872425 ; accuracy 0.6059333
epoch 15 : loss 1.4821132 ; accuracy 0.60775
epoch 16 : loss 1.4770155 ; accuracy 0.60973334
epoch 17 : loss 1.4719479 ; accuracy 0.6116833
epoch 18 : loss 1.4669101 ; accuracy 0.61396664
epoch 19 : loss 1.4619026 ; accuracy 0.61615
epoch 20 : loss 1.4569253 ; accuracy 0.61828333
epoch 21 : loss 1.4519774 ; ac